# Tiny Transformer Quantization Demo
Show how to quantize a mini-GPT stack, run generation, and highlight how packed ternary GEMMs shrink memory while surfacing hardware-ready chunks.


## Environment setup
```bash
pip install .[torch]
```
These notebooks assume `t81lib` is installed from the repository so the Python bindings line up with the C++ core.


In [ ]:
import math
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

import t81lib

plt.style.use('seaborn-v0_8-colorblind')


## Model structure
We build a micro transformer (64-dimensional embeddings, 2 blocks, 4 heads) and reuse the same linear layers for both float and ternary-inference paths.


In [ ]:
def build_tryte_table():
    table = np.empty(27, dtype=np.uint8)
    for t2 in (-1, 0, 1):
        for t1 in (-1, 0, 1):
            for t0 in (-1, 0, 1):
                idx = (t0 + 1) + 3 * (t1 + 1) + 9 * (t2 + 1)
                value = (t0 + 3 * t1 + 9 * t2) + 13
                table[idx] = value
    return table

TRITS_TO_TRYTE = build_tryte_table()

def pack_columnwise(trits: np.ndarray, k_limbs: int, k_actual: int) -> np.ndarray:
    cols = trits.shape[1]
    padded = np.zeros((k_limbs * 48, cols), dtype=np.int8)
    padded[:k_actual, :] = trits[:k_actual, :]
    packed = np.zeros((k_limbs, cols), dtype=np.dtype('V16'))
    view = packed.view(np.uint8).reshape(-1, 16)
    cursor = 0
    for limb_idx in range(k_limbs):
        chunk = padded[limb_idx * 48 : (limb_idx + 1) * 48]
        triples = chunk.reshape(16, 3, cols)
        for col in range(cols):
            for tryte_idx in range(16):
                triple = triples[tryte_idx, :, col] + 1
                tri_index = triple[0] + 3 * triple[1] + 9 * triple[2]
                view[cursor, tryte_idx] = TRITS_TO_TRYTE[tri_index]
            cursor += 1
    return packed

class QuantizedLinearInference:
    def __init__(self, module: nn.Linear, threshold: float = 0.33):
        weight = module.weight.detach().cpu().numpy().astype(np.float32)
        self.weight_packed = t81lib.pack_dense_matrix(weight, threshold=threshold)
        self.k_limbs = self.weight_packed.shape[1]
        self.k_actual = weight.shape[1]
        self.rows = weight.shape[0]
        self.bias = module.bias.detach().cpu().numpy() if module.bias is not None else None
        self.threshold = threshold

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        batch = inputs.shape[0] * inputs.shape[1] if inputs.ndim == 3 else inputs.shape[0]
        flat = inputs.reshape(-1, self.k_actual).detach().cpu().numpy().astype(np.float32)
        trits = t81lib.quantize_to_trits(flat, threshold=self.threshold)
        packed_rhs = pack_columnwise(trits.T, self.k_limbs, self.k_actual)
        output = np.zeros((self.rows * flat.shape[0],), dtype=np.float32)
        t81lib.gemm_ternary(
            self.weight_packed,
            packed_rhs,
            output,
            self.rows,
            flat.shape[0],
            self.k_limbs * 48,
        )
        output = output.reshape(self.rows, flat.shape[0]).T
        if self.bias is not None:
            output += self.bias.reshape(1, -1)
        return torch.from_numpy(output.reshape(inputs.shape[0], inputs.shape[1], self.rows) if inputs.ndim == 3 else output)


In [ ]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.lin1 = nn.Linear(dim, hidden_dim)
        self.lin2 = nn.Linear(hidden_dim, dim)

    def forward(self, x):
        return self.lin2(F.gelu(self.lin1(x)))

class TransformerBlock(nn.Module):
    def __init__(self, dim, heads, ff_dim):
        super().__init__()
        self.qkv_proj = nn.Linear(dim, dim * 3)
        self.out_proj = nn.Linear(dim, dim)
        self.ffn = FeedForward(dim, ff_dim)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.heads = heads
        self.dim = dim

    def forward(self, x):
        residual = x
        B, T, C = x.shape
        qkv = self.qkv_proj(x).view(B, T, 3, self.heads, C // self.heads)
        q, k, v = qkv.unbind(dim=2)
        attn_scores = (q @ k.transpose(-2, -1)) / math.sqrt(C // self.heads)
        attn = F.softmax(attn_scores, dim=-1)
        attn_out = (attn @ v).reshape(B, T, C)
        x = self.norm1(residual + self.out_proj(attn_out))
        x = self.norm2(x + self.ffn(x))
        return x

class MiniTransformer(nn.Module):
    def __init__(self, vocab_size=64, dim=64, heads=4, layers=2, ff_dim=256, seq_len=16):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, dim)
        self.pos_emb = nn.Parameter(torch.randn(seq_len, dim))
        self.blocks = nn.ModuleList([TransformerBlock(dim, heads, ff_dim) for _ in range(layers)])
        self.norm = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, vocab_size)

    def forward(self, tokens):
        B, T = tokens.shape
        x = self.token_emb(tokens) + self.pos_emb[:T]
        for block in self.blocks:
            x = block(x)
        logits = self.head(self.norm(x))
        return logits

model = MiniTransformer()
model.eval()
print('Mini transformer architecture ready')


In [ ]:
batch_tokens = torch.randint(0, 64, (2, 16))
float_logits = model(batch_tokens)
print('Float logits shape', float_logits.shape)


In [ ]:
threshold = 0.33
quantized_layers = {}
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        quantized_layers[name] = QuantizedLinearInference(module, threshold=threshold)
print('Quantized linear map entry count', len(quantized_layers))


In [ ]:
def lookup_quantized(name):
    return quantized_layers[name]

def quantized_forward(tokens: torch.Tensor) -> torch.Tensor:
    B, T = tokens.shape
    x = model.token_emb(tokens) + model.pos_emb[:T]
    for i, block in enumerate(model.blocks):
        residual = x
        qkv = lookup_quantized(f'blocks.{i}.qkv_proj').forward(x)
        qkv = qkv.view(B, T, 3, block.heads, block.dim // block.heads)
        q, k, v = qkv.unbind(dim=2)
        attn_scores = (q @ k.transpose(-2, -1)) / math.sqrt(block.dim // block.heads)
        attn = F.softmax(attn_scores, dim=-1)
        attn_out = (attn @ v).reshape(B, T, block.dim)
        projected = lookup_quantized(f'blocks.{i}.out_proj').forward(attn_out)
        x = block.norm1(residual + projected.reshape(B, T, block.dim))
        ffn = lookup_quantized(f'blocks.{i}.ffn.lin1').forward(x)
        ffn = F.gelu(ffn)
        ffn_out = lookup_quantized(f'blocks.{i}.ffn.lin2').forward(ffn)
        x = block.norm2(x + ffn_out)
    logits = lookup_quantized('head').forward(model.norm(x))
    return logits


In [ ]:
def sample_sequence(prefix, forward_fn, steps=10):
    ids = list(prefix)
    for _ in range(steps):
        tokens = torch.tensor([ids], dtype=torch.long)
        logits = forward_fn(tokens)
        probs = F.softmax(logits[:, -1, :], dim=-1)
        next_token = torch.multinomial(probs, num_samples=1).item()
        ids.append(next_token)
    return ids

alphabet = 'abcdefghijklmnopqrstuvwxyz '# 26 lowercase + space
prefix = [0, 1, 2, 3]
float_seq = sample_sequence(prefix, lambda tokens: model(tokens), steps=6)
quant_seq = sample_sequence(prefix, quantized_forward, steps=6)

def decode(seq):
    return ''.join(alphabet[i % len(alphabet)] for i in seq)

print('Float generation', decode(float_seq))
print('Ternary generation', decode(quant_seq))


In [ ]:
def timed_forward(fn, steps=5):
    tokens = torch.randint(0, 64, (2, 16))
    start = time.perf_counter()
    for _ in range(steps):
        fn(tokens)
    return (time.perf_counter() - start) / steps

float_latency = timed_forward(lambda tokens: model(tokens))
ternary_latency = timed_forward(quantized_forward)
print(f'Float latency {float_latency:.4f}s, ternary latency {ternary_latency:.4f}s (per call)')


In [ ]:
float_bytes = sum(m.weight.numel() * 4 for m in model.modules() if isinstance(m, nn.Linear))
ternary_bytes = sum(layer.weight_packed.nbytes for layer in quantized_layers.values())
binary_bits = sum(m.weight.numel() for m in model.modules() if isinstance(m, nn.Linear))
binary_bytes = math.ceil(binary_bits / 8)
print(f'Float linear params {float_bytes / 1024:.1f}kB, ternary packed {ternary_bytes / 1024:.1f}kB (~{ternary_bytes / binary_bytes * 100:.1f}% of 1-bit)')

labels = ['Float32', 'Ternary (pack)']
size_vals = [float_bytes, ternary_bytes]
latency_vals = [float_latency, ternary_latency]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.bar(labels, [val / 1024 for val in size_vals], color=['#1f77b4', '#ff7f0e'])
ax1.set_ylabel('Memory (kB)')
ax1.set_title('Linear layer footprint')
ax2.bar(labels, latency_vals, color=['#2ca02c', '#d62728'])
ax2.set_ylabel('Latency (s)')
ax2.set_title('Per forward call (averaged)')
fig.tight_layout()
plt.show()


In [ ]:
values = [1, -1, 1]
row_idx = [0, 0, 1]
col_idx = [0, 1, 2]
B = np.random.rand(3, 8).astype(np.float32)
C = np.zeros((2 * 8,), dtype=np.float32)
t81lib.spmm_simple(values, row_idx, col_idx, 2, 3, B, C, 8)
C = C.reshape(2, 8)
print('Sparse attention replay result', C)


## Reflection and extensibility
Packed ternary GEMMs run inside the same training-friendly PyTorch graph but keep the inference kernels deterministic and portable.
Use `QuantizedLinearInference` for each projection in larger models (blending through `t81.torch` or by caching quantized layers) and hook into `t81lib.gemm_ternary` + `spmm_simple` to test custom sparsity budgets or simulated accelerator fabrics.
To scale to Llama-style stacks, quantize projection weights once, pack them into `pack_dense_matrix`, and schedule the packed buffers to your triplet-aware GEMM engine; converting your existing float checkpoints before conversion keeps evaluation reproducible.
